In [11]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import spatial
from sklearn.manifold import TSNE
from tqdm.notebook import tqdm

from scipy.spatial.distance import cosine
from pprint import pprint

# Keywords

In [2]:
conflict = ['war', 'fight', 'death', 'hostility', 'bomb', 'battle', 'nazi', 'army']
politics = ['politics', 'debate', 'election', 'government', 'democracy']
fashion = ['fashion', 'model', 'magazine', 'walk', 'glamour', 'outfit']
culture = ['culture', 'opera', "festival", 'collection', 'exhibition', 'art', 'museum']
sports = ['sport', 'medal', 'game', 'championship', 'club', 'score', 'play']
education = ['education', 'professor', 'study', 'research', 'university']
# football = []
# other

descriptive_keywords = [conflict, politics, fashion, culture, sports, education]
descriptive_keywords
# How to avoid words with high tf?

[['war', 'fight', 'death', 'hostility', 'bomb', 'battle', 'nazi', 'army'],
 ['politics', 'debate', 'election', 'government', 'democracy'],
 ['fashion', 'model', 'magazine', 'walk', 'glamour', 'outfit'],
 ['culture', 'opera', 'festival', 'collection', 'exhibition', 'art', 'museum'],
 ['sport', 'medal', 'game', 'championship', 'club', 'score', 'play'],
 ['education', 'professor', 'study', 'research', 'university']]

# Word Embedding

In [4]:
GLOVE_LOCATION = '../../../../glove.42B.300d.txt'
embeddings_dict = {}

### Loading GLoVe

In [9]:
%%time 
# ~3 minutes to load

num_lines = sum(1 for line in open(GLOVE_LOCATION,'r', encoding="utf-8"))

with open(GLOVE_LOCATION, 'r', encoding="utf-8") as f:
     for line in tqdm(f, total=num_lines):
        values = line.split()
        token = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[token] = vector

  0%|          | 0/1917495 [00:00<?, ?it/s]

CPU times: total: 2min 30s
Wall time: 2min 33s


### Example paragraphs + lemmatised wordlists

In [125]:
textstring1 = "Napoleon as Mars the Peacemaker had its inception after Canova was hired to make a bust of Napoleon in 1802. The statue was begun in 1802, with Napoleon requesting to be shown in a French General's uniform, Canova rejected this, insisting on an allusion to Mars, the Roman god of War. It was completed in 1806. In 1811, the statue arrived in Paris, but not installed; neither was its bronze copy in the Foro Napoleonico in Milan. In 1815, the original went to the Duke of Wellington, after his victory at Waterloo against Napoleon. "
textstring2 = "is only red card with Manchester United came on 6 January 2000, in the 2000 FIFA Club World Championship against Necaxa. During his time at Real Madrid, he amassed 41 yellow cards and four red cards in La Liga; he also received a red card in a Copa del Rey match against Valencia on 21 January 2004, in Madrid. His only red card with LA Galaxy came on 15 August 2009, in a 2–0 home defeat to Seattle Sounders in the MLS. He received one red card while at Paris Saint-Germain – picked up in injury time – in a match against Evian on 28 April 2013. Between 2000 and 2013, Beckham played 572 competitive games for England, Milan, LA Galaxy, Manchester United, Real Madrid and PSG, and he received nine red cards – one every 63–64 matches, on average. "
textstring3 = "David Robert Joseph Beckham (; born 2 May 1975) is an English former professional footballer, the current president &amp; co-owner of Inter Miami CF and co-owner of Salford City. He played for Manchester United, Preston North End (on loan), Real Madrid, AC Milan (on loan), LA Galaxy, Paris Saint-Germain and the England national team, for which he held the appearance record for an outfield player until 2016. He is the first English player to win league titles in four countries: England, Spain, the United States and France. He retired in May 2013 after a 20-year career, during which he won 19 major trophies."
textstring4 = "In 1872, Abbott went abroad to study with Antonio Sangiovanni in Milan. This was followed by further studies with Mathilde Marchesi, Pierre François Wartel and Enrico Delle Sedie in Paris. She appeared in several productions in Paris, earning rave reviews for her fine soprano voice. She was awarded a contract with the Royal Opera in London and made her début at Covent Garden as Marie in 'La Fille du régiment' in 1876. However, her contract was cancelled shortly thereafter when she refused to sing Violetta from Verdi's 'La Traviata' on moral grounds. That same year she secretly married Eugene Wetherell (d. 1889) and they returned to the United States, where she remained for the rest of her life."

In [23]:
words = ['peacemaker', 'inception', 'hire', 'bust', 'statue', 'begin', 'request', 'show', 'french', 'general', 'uniform', 'reject', 'insist', 'allusion', 'roman', 'war', 'complete', 'statue', 'arrive', 'instal', 'bronze', 'copy', 'original', 'go', 'victory']
words2 = ['red', 'card', 'come', 'club', 'world', 'championship', 'necaxa', 'time', 'real', 'amass', 'yellow', 'card', 'red', 'card', 'receive', 'red', 'card', 'red', 'card', 'la', 'galaxy', 'come', 'home', 'defeat', 'sounder', 'receive', 'red', 'card', 'pick', 'injury', 'time', 'match', 'play', 'competitive', 'game', 'receive', 'red', 'card', 'match', 'average']
words3 = ['bear', 'english', 'professional', 'footballer', 'current', 'president', 'amp', 'co', 'owner', 'co', 'owner', 'play', 'loan', 'loan', 'hold', 'appearance', 'record', 'outfield', 'player', 'english', 'player', 'win', 'league', 'title', 'country', 'retire', 'year', 'career', 'win', 'major', 'trophy']
words4 = ['go', 'study', 'follow', 'study', 'sedie', 'appear', 'production', 'earn', 'rave', 'review', 'fine', 'soprano', 'voice', 'award', 'contract', 'royal', 'opera', 'début', 'covent', 'garden', 'régiment', 'contract', 'cancel', 'refuse', 'sing', 'violetta', 'moral', 'ground', 'year', 'marry', 'return', 'remain', 'rest', 'life']

### Algorithm

In [21]:
l = ['a', 'b', 'c']
l[:500]

['a', 'b', 'c']

In [113]:
%%time


# parameters 
number_of_keywords = 2 # setting it to 0 will use them all
bottom_threshold = 0.1
verbose1, verbose2 = 0, 0

keywords_to_use = number_of_keywords if number_of_keywords else 1000
similar_categories = []

for word in words:
    if verbose1 or verbose2:
        print(f"word: \t\t'{word}'")
        
    closeness = []
    
    for category in descriptive_keywords:
        summed_similarity = 0
        for keyword in category[:keywords_to_use]:
            keyword_vector = embeddings_dict[keyword]        
            word_vector = embeddings_dict[word]
            value = 1 - cosine(keyword_vector, word_vector)
            summed_similarity += value
        
        normalized_similarity = summed_similarity/len(category[:keywords_to_use])
        closeness.append((normalized_similarity, category[0]))
        
        if verbose1:
            # print('___________________________')
            print('===>', '\t\tcategory:', category[0], '\n\t\tsimilarity:', normalized_similarity, f"\n\t\tkeywords: {category[:keywords_to_use]}\n")
    
     
    similar_category = max(closeness)
    
    sortedcat = sorted(closeness, key=lambda item: item[0], reverse=True)
    if (sortedcat[0][0] - 0.05) > sortedcat[2][0]:
        allowed = True
    else:
        allowed = False
    

    
    if similar_category[0] > bottom_threshold and allowed:
        similar_categories.append((word, similar_category))
        if verbose2:
            #print('category similarity:')
            # pprint(sorted(closeness, key=lambda x: x[0], reverse=True))
            print(f"choice: \tkept")
            print(f"\n==> \tcategory:, {similar_category[1]}, \n\tsimilarity score:, {similar_category[0]}")
    elif verbose2:
        print(f"choice: \tdiscarded")
        reason = 'ambiguity' if not allowed else 'low similarity score'
        print(f"reasoning: \t{reason}")
    
    if verbose2:
        # print(f"analysis: {'not' if not allowed else ''} enough difference\nscores:")
        print("\nscores:\n\t----category----          ----score----")
        for i in sortedcat:
            print(f"\t{i[1]:<10s} \t\t{i[0]}")
        print()

        print('='*100)
        print('')

# print('\nwords:', words, '\n')

# len(similar_categories), similar_categories

CPU times: total: 31.2 ms
Wall time: 18 ms


In [193]:
def categorize_text(words=[], number_of_keywords=1, bottom_threshold=0.1, verbose1=False, verbose2=False):
    # parameters 
    # number_of_keywords = 2 # setting it to 0 will use them all
    # bottom_threshold = 0.1
    # verbose1, verbose2 = 0, 0

    keywords_to_use = number_of_keywords if number_of_keywords else 1000
    similar_categories = []

    for word in words:
        try:
            word_vector = embeddings_dict[word]
        except:
            continue
            
        if verbose1 or verbose2:
            print(f"word: \t\t'{word}'")

        closeness = []

        for category in descriptive_keywords:
            summed_similarity = 0
            for keyword in category[:keywords_to_use]:
                keyword_vector = embeddings_dict[keyword]
                value = 1 - cosine(keyword_vector, word_vector)
                summed_similarity += value
            
            normalized_similarity = summed_similarity/len(category[:keywords_to_use])
            closeness.append((normalized_similarity, category[0]))

            if verbose1:
                # print('___________________________')
                print('===>', '\t\tcategory:', category[0], '\n\t\tsimilarity:', normalized_similarity, f"\n\t\tkeywords: {category[:keywords_to_use]}\n")


        similar_category = max(closeness)

        sortedcat = sorted(closeness, key=lambda item: item[0], reverse=True)
        if (sortedcat[0][0] - 0.05) > sortedcat[2][0]:
            allowed = True
        else:
            allowed = False



        if similar_category[0] > bottom_threshold and allowed:
            similar_categories.append((word, similar_category))
            if verbose2:
                #print('category similarity:')
                # pprint(sorted(closeness, key=lambda x: x[0], reverse=True))
                print(f"choice: \tkept")
                print(f"\n==> \tcategory:, {similar_category[1]}, \n\tsimilarity score: {similar_category[0]}")
        elif verbose2:
            print(f"choice: \tdiscarded")
            reason = 'ambiguity' if not allowed else 'low similarity score'
            print(f"reasoning: \t{reason}")

        if verbose2:
            # print(f"analysis: {'not' if not allowed else ''} enough difference\nscores:")
            print("\nscores:\n\t----category----          ----score----")
            for i in sortedcat:
                print(f"\t{i[1]:<10s} \t\t{i[0]}")
            print()

            print('='*100)
            print('')
    
    categories_dict = {key[0]: 0 for key in descriptive_keywords}
    
    for x in similar_categories:
        categories_dict[x[1][1]] += x[1][0] #print(x[1])
    

    results = list(sorted(categories_dict.items(), key=lambda item: item[1], reverse=True))
    pprint(results)

    print(f"\nThe dominant category is: '{results[0][0]}'", end='')
    if (results[0][1] - (float(results[0][1])/5)) <= results[1][1]:  
        print(f", closely followed by: '{results[1][0]}'.")
    if verbose2:
        print('\n')
        pprint(similar_categories)
    print('\n --------------------------------------------------------------------')
    
    return results[0][0]
# print('\nwords:', words, '\n')

# len(similar_categories), 

In [129]:
category = categorize_text(words=words4, number_of_keywords=1, bottom_threshold=0.1, verbose1=False, verbose2=False)
textstring4

[('education', 3.663667678833008),
 ('war', 1.2850384712219238),
 ('fashion', 0.997096598148346),
 ('culture', 0.9013673067092896),
 ('politics', 0),
 ('sport', 0)]

The dominant category is: 'education'

"In 1872, Abbott went abroad to study with Antonio Sangiovanni in Milan. This was followed by further studies with Mathilde Marchesi, Pierre François Wartel and Enrico Delle Sedie in Paris. She appeared in several productions in Paris, earning rave reviews for her fine soprano voice. She was awarded a contract with the Royal Opera in London and made her début at Covent Garden as Marie in 'La Fille du régiment' in 1876. However, her contract was cancelled shortly thereafter when she refused to sing Violetta from Verdi's 'La Traviata' on moral grounds. That same year she secretly married Eugene Wetherell (d. 1889) and they returned to the United States, where she remained for the rest of her life."

# paragraphs

In [134]:
import pickle
import os 
from tqdm.notebook import tqdm
import re
import spacy

# nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_lg")
nlp.max_length = 1500000
nlp.disable_pipes('ner', 'parser')

['ner', 'parser']

In [136]:
FILE_PATH = "../../../../data/enwiki_city_pairs/biggest_cities_5/paris_milan.txt"
POS = ["VERB", "NOUN", "ADJ"]

with open(FILE_PATH, 'r', encoding='utf-16') as f:
    city_pair_text_list = [x.strip().lower() for x in f.read().replace('"', "'").replace('“', "'").replace('”', "'").split('\n') if len(x) and 'title=' not in x]

sample_of_paragraphs = city_pair_text_list[:50]
# for x in city_pair_text_list[:50]:
#     print(x)
#     print()

# nr_of_chunks = len(city_pair_text_list)//1000 + 1
# chunk_size = (len(city_pair_text_list)-1)//nr_of_chunks
# chunked_text = [' '.join(city_pair_text_list[offs:offs+chunk_size]) for offs in range(0, len(city_pair_text_list), chunk_size)]

chunked_text = sample_of_paragraphs
processed_text = [text for text in tqdm(nlp.pipe(chunked_text, n_process=2, batch_size=1, disable=["ner", "parser"]), total=len(chunked_text))]
lemmatized_text = [[word.lemma_ for word in text if word.pos_ in POS and not word.is_punct and not word.is_stop] for text in processed_text]
# [word.lemma_ for word in text if word.pos_ in ["VERB"] and not word.is_punct and not word.is_stop]
# # [word.lemma_ for word in processed_text if word.pos_ in ["VERB"] and not word.is_punct and not word.is_stop]
regexed_text = [[re.sub(r'\W+', '', word) for word in text] for text in lemmatized_text]
# flattened_words = [item for sublist in regexed_text for item in sublist]

  0%|          | 0/50 [00:00<?, ?it/s]

In [157]:
%%time

categories = []
print('\n --------------------------------------------------------------------')
for index, paragraph in enumerate(regexed_text):
    print(sample_of_paragraphs[index])
    print()
    category = categorize_text(words=paragraph, number_of_keywords=1, bottom_threshold=0.1, verbose1=False, verbose2=False)
    categories.append(category)


 --------------------------------------------------------------------
there are plenty of air connections between yerevan and other regional cities, including athens, barcelona, beirut, berlin, bucharest, brussels, damascus, doha, dubai, istanbul, kyiv, kuwait city, london, milan, minsk, moscow, paris, prague, riga, rome, tehran, tel-aviv, tbilisi, vienna, venice and warsaw, as well as daily connections to most major cities within the cis region. statistics show that the number of tourists arriving in the country by air transportation increases yearly. in 2018, passenger flow at the two main airports of armenia reached a record high of 2,856,673 million people. in december 2019, yearly passenger flow exceeded 3,000,000 million people for the first time in armenia's history.

[('education', 4.481926411390305),
 ('culture', 3.6081653833389282),
 ('war', 1.7753939628601074),
 ('sport', 0.6986993551254272),
 ('politics', 0),
 ('fashion', 0)]

The dominant category is: 'education', closely

[('education', 5.094779193401337),
 ('war', 4.153684765100479),
 ('culture', 1.3112236559391022),
 ('fashion', 0.6964422464370728),
 ('politics', 0.4467393755912781),
 ('sport', 0)]

The dominant category is: 'education', closely followed by: 'war'.

 --------------------------------------------------------------------
after his election, he made luigi maglione his successor as cardinal secretary of state. cardinal maglione, a seasoned vatican diplomat, had reestablished diplomatic relations with switzerland and was for many years nuncio in paris. yet, maglione did not exercise the influence of his predecessor pacelli, who as pope continued his close relation with monsignors montini (later pope paul vi) and domenico tardini. after the death of maglione in 1944, pius left the position open and named tardini head of its foreign section and montini head of the internal section. tardini and montini continued serving there until 1953, when pius xii decided to appoint them cardinals, an hono

In [160]:
from collections import Counter
Counter(categories)

Counter({'education': 33,
         'war': 6,
         'culture': 4,
         'sport': 2,
         'fashion': 4,
         'politics': 1})

# TF-iDF

In [162]:
import pandas as pd

In [164]:
TFIDF_PATH = '../../../../data/word_frequencies/tfidf_nouns_real_words.csv'

tfidf = pd.read_csv(TFIDF_PATH)
tfidf.set_index('word', inplace=True)

In [198]:
tfidf_list = tfidf['london_berlin']
#tfidf_list.sort_valu

In [199]:
corrected_tfidf_list = tfidf_list[tfidf_list != 0.000000].sort_values(ascending=False)
corrected_tfidf_list

word
marathon     0.001395
film         0.001380
album        0.001019
specimen     0.000906
band         0.000877
               ...   
defender     0.000006
itinerary    0.000006
scoring      0.000006
double       0.000005
aggregate    0.000004
Name: london_berlin, Length: 9168, dtype: float64

In [200]:
sample_words = list(corrected_tfidf_list.index)[:100]

In [201]:
sample_words

['marathon',
 'film',
 'album',
 'specimen',
 'band',
 'art',
 'orchestra',
 'song',
 'symphony',
 'war',
 'berlin',
 'economic',
 'correspondent',
 'fashion',
 'concerto',
 'museum',
 'artist',
 'recording',
 'label',
 'woman',
 'story',
 'video',
 'collection',
 'hospital',
 'nazi',
 'office',
 'book',
 'ballet',
 'news',
 'painting',
 'intelligence',
 'documentary',
 'studio',
 'music',
 'gallery',
 'producer',
 'novel',
 'friend',
 'dance',
 'opera',
 'prey',
 'violin',
 'piano',
 'portrait',
 'release',
 'college',
 'festival',
 'father',
 'ensemble',
 'race',
 'publication',
 'family',
 'treaty',
 'child',
 'musician',
 'medicine',
 'life',
 'editor',
 'newspaper',
 'magazine',
 'chorus',
 'writer',
 'play',
 'design',
 'fellowship',
 'exhibition',
 'feature',
 'actress',
 'jazz',
 'raid',
 'government',
 'society',
 'bird',
 'zoo',
 'scene',
 'physician',
 'ambassador',
 'sound',
 'embassy',
 'protest',
 'wall',
 'award',
 'bomber',
 'rabbi',
 'graffiti',
 'record',
 'piece',
 '

In [202]:
categorize_text(words=sample_words, number_of_keywords=1, bottom_threshold=0.1, verbose1=False, verbose2=False)

[('fashion', 8.52088913321495),
 ('war', 7.003579199314117),
 ('culture', 6.03248992562294),
 ('education', 5.799533396959305),
 ('politics', 2.067651242017746),
 ('sport', 0.8457217216491699)]

The dominant category is: 'fashion', closely followed by: 'war'.

 --------------------------------------------------------------------


'fashion'